In [27]:
import cv2
import torch
import requests
from ultralytics import YOLO

model = YOLO('C:/Users/dell/OneDrive/Bureau/mastere2/projet_fin_etude/runs/detect/train9/weights/best.pt')
url = 'http://<IP_de_votre_ESP32>:81/stream'
server_url = 'http://<IP_de_votre_serveur>:5000/upload'

cap = cv2.VideoCapture(url)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    detections = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy)
            detections.append({'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2})
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Envoyer les résultats au serveur
    response = requests.post(server_url, json={'detections': detections})

    cv2.imshow('ESP32-CAM Stream', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
